In [8]:
!pip install boto3 botocore torch-model-archiver


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [21]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

# print(aws_access_key_id)
# print(bucket_name)
# print(endpoint_url)
# print(region_name)
# print(aws_secret_access_key)

In [10]:
work_dir=os.getcwd()
print(f"Current working directory: {work_dir}")

Current working directory: /opt/app-root/src/stable-diffusion-model


In [5]:
tuned_model_dir = f"{work_dir}/diffusion_model"
print(f"Local tuned model dir: {tuned_model_dir}")

Local tuned model dir: /opt/app-root/src/stable-diffusion-model/diffusion_model


In [12]:
minio_console=!oc get route minio-ui -o jsonpath='{.spec.host}'
print(f"Minio console: https://{minio_console.n}/browser/{bucket_name}")

Minio console: https://minio-ui-worldline.apps.ai-dev02.kni.syseng.devcluster.openshift.com/browser/stable-diffusion


In [13]:
# This can take 10-15mins to zip up

!(cd {tuned_model_dir} && zip -r {work_dir}/model/model.zip *)

updating: feature_extractor/ (stored 0%)
updating: feature_extractor/preprocessor_config.json (deflated 48%)
updating: model_index.json (deflated 56%)
updating: safety_checker/ (stored 0%)
updating: safety_checker/model.safetensors (deflated 8%)
updating: safety_checker/config.json (deflated 49%)
updating: scheduler/ (stored 0%)
updating: scheduler/scheduler_config.json (deflated 41%)
updating: text_encoder/ (stored 0%)
updating: text_encoder/model.safetensors (deflated 8%)
updating: text_encoder/config.json (deflated 43%)
updating: tokenizer/ (stored 0%)
updating: tokenizer/special_tokens_map.json (deflated 73%)
updating: tokenizer/vocab.json (deflated 71%)
updating: tokenizer/tokenizer_config.json (deflated 64%)
updating: tokenizer/merges.txt (deflated 60%)
updating: unet/ (stored 0%)
updating: unet/diffusion_pytorch_model.safetensors (deflated 8%)
updating: unet/config.json (deflated 63%)
updating: vae/ (stored 0%)
updating: vae/diffusion_pytorch_model.safetensors (deflated 8%)
upda

In [14]:
!(cd {work_dir}/model && torch-model-archiver --model-name stable-diffusion --version 1.0 --handler stable_diffusion_handler.py --extra-files model.zip -r requirements.txt -f)

In [22]:
if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [23]:
list_objects("models")

In [25]:
model_dir = work_dir + "/model"

if not os.path.isdir(model_dir):
    raise ValueError(f"The directory '{model_dir}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")

num_files = upload_directory_to_s3(model_dir, "models")

if num_files == 0:
    raise ValueError(f"No files uploaded. Check the {model_dir} directory.")

/opt/app-root/src/stable-diffusion-model/model/stable_diffusion_handler.py -> models/stable_diffusion_handler.py
/opt/app-root/src/stable-diffusion-model/model/requirements.txt -> models/requirements.txt
/opt/app-root/src/stable-diffusion-model/model/stable-diffusion.mar -> models/stable-diffusion.mar
/opt/app-root/src/stable-diffusion-model/model/config.properties -> models/config.properties
/opt/app-root/src/stable-diffusion-model/model/model.zip -> models/model.zip
/opt/app-root/src/stable-diffusion-model/model/.ipynb_checkpoints/requirements-checkpoint.txt -> models/.ipynb_checkpoints/requirements-checkpoint.txt
/opt/app-root/src/stable-diffusion-model/model/.ipynb_checkpoints/config-checkpoint.properties -> models/.ipynb_checkpoints/config-checkpoint.properties
/opt/app-root/src/stable-diffusion-model/model/.ipynb_checkpoints/stable_diffusion_handler-checkpoint.py -> models/.ipynb_checkpoints/stable_diffusion_handler-checkpoint.py


In [26]:
list_objects("models")

models/.ipynb_checkpoints/config-checkpoint.properties
models/.ipynb_checkpoints/requirements-checkpoint.txt
models/.ipynb_checkpoints/stable_diffusion_handler-checkpoint.py
models/config.properties
models/model.zip
models/requirements.txt
models/stable-diffusion.mar
models/stable_diffusion_handler.py
